In [1]:
%load_ext pycodestyle_magic
# Activamos las alertas de estilo
%pycodestyle_on

import requests
import whois
import builtwith
import time
import re
import pandas as pd
from bs4 import BeautifulSoup


# Esta es la web donde aparecen las películas ganadoras y nominadas a mejor película en
# los óscar durante la década de 2010's
url = "https://www.filmaffinity.com/es/awards3.php?award_id=academy_awards&decade=2010"

# Modificamos el User-Agent para evitar posibles problemas
headers = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, sdch, br",
"Accept-Language": "en-US,en;q=0.8",
"Cache-Control": "no-cache",
"dnt": "1",
"Pragma": "no-cache",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"
}

In [2]:
builtwith.builtwith(url)

{'programming-languages': ['PHP'],
 'web-servers': ['Apache'],
 'analytics': ['comScore'],
 'javascript-frameworks': ['jQuery', 'jQuery UI']}

In [3]:
print(whois.whois(url))

{
  "domain_name": [
    "FILMAFFINITY.COM",
    "filmaffinity.com"
  ],
  "registrar": "Arsys Internet, S.L. dba NICLINE.COM",
  "whois_server": "whois.nicline.com",
  "referral_url": null,
  "updated_date": [
    "2020-06-21 07:21:16",
    "2015-01-13 15:24:07"
  ],
  "creation_date": "2001-06-20 14:23:27",
  "expiration_date": "2021-06-20 14:23:27",
  "name_servers": [
    "NS1.FILMAFFINITY.COM",
    "NS2.FILMAFFINITY.COM"
  ],
  "status": [
    "ok https://icann.org/epp#ok",
    "ok https://www.icann.org/epp#ok"
  ],
  "emails": [
    "email@nicline.com",
    "whoiscontact@domainconnection.info"
  ],
  "dnssec": [
    "unsigned",
    "Unsigned"
  ],
  "name": "REDACTED FOR PRIVACY",
  "org": null,
  "address": "REDACTED FOR PRIVACY",
  "city": "REDACTED FOR PRIVACY",
  "state": "Madrid",
  "zipcode": "REDACTED FOR PRIVACY",
  "country": "ES"
}


In [2]:
# Se descarga la web y se comprueba si ha habido algún error.
main_page = requests.get(url, headers=headers)
if main_page.status_code == 200:
    main_soup = BeautifulSoup(main_page.content)
    print(main_soup.prettify())
else:
    print("No se ha podidos obtener el DOM.")

<!DOCTYPE html>
<html lang="es">
 <head>
  <title>
   Premios Oscars - Década 2010 (Nominadas a mejor película de cada año) - FilmAffinity
  </title>
  <meta charset="utf-8"/>
  <meta content="Filmaffinity, tu página personalizada de votación y recomendación de películas y series" name="description"/>
  <meta content="FilmAffinity" property="og:site_name"/>
  <meta name="twitter:site" value="@Filmaffinity"/>
  <meta content="https://www.filmaffinity.com/es/awards3.php?award_id=academy_awards&amp;decade=2010" property="og:url"/>
  <meta content="FilmAffinity" property="og:title"/>
  <meta content="" property="og:description"/>
  <meta content="" property="og:image"/>
  <link href="http://www.filmaffinity.com/es/awards3.php?award_id=academy_awards&amp;decade=2010" rel="canonical"/>
  <link href="https://m.filmaffinity.com/es/awardsdecade.php?award_id=academy_awards&amp;decade=2010" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/favicon.png" rel="icon" type="i

In [3]:
# Se crea una lista donde se almacenarán las URL de las películas
URL_films = []

# Dentro del div con el atributo class especificado se encuentran las
# ganadoras del premio
awarded_films = main_soup.body.find_all("div", class_="aw-mc2 big-poster\
 winner-border")

# Dentro del div con este atributo class se encuentran las películas
# que han sido nominadas pero no ganaron el premio
nominated_films = main_soup.body.find_all("div", class_="aw-mc2 big-poster")

# Observando su estructura, la URL a la web con toda la información
# de la película se encuentra dentro de la etiqueta a, en el atributo
# href, por lo que iteramos para almacenarlas en URL_films
for i in range(len(awarded_films)):
    URL_films.append(awarded_films[i].a.get("href"))
for i in range(len(nominated_films)):
    URL_films.append(nominated_films[i].a.get("href"))

In [9]:
def get_page(url):
    try:
        soup = None
        t0 = time.time()
        page = requests.get(url, headers=headers)
        delay = time.time() - t0
        if page.status_code == 200:
            soup = BeautifulSoup(page.content)
        else:
            print("Ha habido algún problema al descargar el contenido.\n")
        time.sleep(10*delay)
    except requests.exceptions.Timeout:
        pass
    except requests.exceptions.RequestException:
        pass
    return soup

In [10]:
def find_movie_info(movie_info):

    wanted_data = ["Título original", "Año", "Duración", "País", "Dirección",
                   "Reparto", "Género", "Sinopsis"]
    dt_list = movie_info.find_all("dt")
    dd_list = movie_info.find_all("dd")

    info = [dd_list[i] for i in range(len(dt_list)) if dt_list[i].
            get_text(strip=True) in wanted_data]

    return info

In [11]:
def get_award(award_info):
    info = []
    for award in award_info.find_all("div"):
        aux = award.get_text(strip=True)
        if not re.search("Mostrar [0-9]{1,2} premios más", aux):
            info.append(aux)
    return info

In [12]:
def load_requests(source_url):
    t0 = time.time()
    r = requests.get(source_url, headers=headers, stream=True)
    delay = time.time() - t0
    if r.status_code == 200:
        aSplit = source_url.split('/')
        ruta = "C:/Users/a091b/images_FA/"+aSplit[len(aSplit)-1]
        print(ruta)
        output = open(ruta, "wb")
        for chunk in r:
            output.write(chunk)
        output.close()
    time.sleep(10*delay)

In [13]:
index = 0
data_dict = {}
for i in URL_films:
    soup = get_page(i)
    # Separamos la web en 3 zonas donde existe información de interés
    movie_info = find_movie_info(soup.find(class_="movie-info"))
    award_info = soup.find(class_="margin-top movie-info").dd
    rating_info = soup.find(id="rat-avg-container").find_all('div')

    # Aquellos atributos que son una lista de valores los generamos previamente
    actor_list = [actor.get_text(strip=True) for actor in movie_info[5].
                  div.find_all("span")[1::2]]
    genre_list = [genre.get_text(strip=True) for genre in movie_info[6].
                  find_all("a")]
    award_list = get_award(award_info)

    # Construimos el diccionario con todos los datos para una película
    data_dict[index] = {
        "title": soup.find("h1", id="main-title").span.get_text(strip=True),
        "original_title": movie_info[0].get_text(strip=True),
        "year": movie_info[1].get_text(strip=True),
        "duration": movie_info[2].get_text(strip=True),
        "country": movie_info[3].get_text(strip=True),
        "director": movie_info[4].div.a.get_text(strip=True),
        "actors": actor_list,
        "genre": genre_list,
        "description": movie_info[7].get_text(strip=True),
        "awards": award_list,
        "average_rating": rating_info[0].get_text(strip=True),
        "rating_votes": rating_info[1].span.get_text(strip=True),
        "poster": soup.find(id="movie-main-image-container").a.get("href")
    }
    index += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


In [45]:
df = pd.DataFrame().from_dict(data_dict, orient='index')
df.head().append(df.tail())

,title,original_title,year,duration,country,director,actors,genre,description,awards,average_rating,rating_votes,poster
0,Green Book,Green Book,2018,130 min.,Estados Unidos,Peter Farrelly,"[Viggo Mortensen, Mahershala Ali, Iqbal Theba,...","[Drama, Comedia, Basado en hechos reales, Come...",Año 1962. Tony Lip (Viggo Mortensen) es un rud...,"[2018: 3 Premios Oscar: Mejor película, guion ...","7,6",38.880,https://pics.filmaffinity.com/green_book-16245...
1,La forma del agua,The Shape of Water,2017,119 min.,Estados Unidos,Guillermo del Toro,"[Sally Hawkins, Doug Jones, Michael Shannon, O...","[Fantástico, Drama, Romance, Thriller, Años 60]",En un inquietante laboratorio de alta segurida...,"[2017: 4 Premios Oscar: Mejor película, direct...","6,5",41.102,https://pics.filmaffinity.com/the_shape_of_wat...
2,Moonlight,Moonlight,2016,111 min.,Estados Unidos,Barry Jenkins,"[Trevante Rhodes, Naomie Harris, Mahershala Al...","[Drama, Infancia, Adolescencia, Acoso escolar,...",Chiron es un joven afroamericano con una difíc...,"[2016: 3 Premios Oscar: Mejor película, guión ...","6,8",28.351,https://pics.filmaffinity.com/moonlight-232276...
3,Spotlight,Spotlight,2015,121 min.,Estados Unidos,Tom McCarthy,"[Mark Ruffalo, Michael Keaton, Rachel McAdams,...","[Drama, Periodismo, Religión, Abusos sexuales,...","En el año 2002, un reducido equipo de reporter...",[2015: 2 Premios Oscar: mejor película y guión...,"7,3",44.623,https://pics.filmaffinity.com/spotlight-726888...
4,Birdman o (La Inesperada Virtud de la Ignorancia),Birdman or (The Unexpected Virtue of Ignorance...,2014,118 min.,Estados Unidos,Alejandro González Iñárritu,"[Michael Keaton, Emma Stone, Edward Norton, Za...","[Comedia, Drama, Teatro, Sátira, Comedia negra...",Después de hacerse famoso interpretando en el ...,[2014: 4 Oscars: incluyendo Mejor película y d...,"7,0",71.165,https://pics.filmaffinity.com/birdman_or_the_u...
84,Un tipo serio,A Serious Man,2009,105 min.,Estados Unidos,Joel Coen,"[Michael Stuhlbarg, Richard Kind, Fred Melamed...","[Comedia, Drama, Religión, Enseñanza, Años 60,...","Medio-oeste americano, 1967. Larry Gopnik (Mic...",[2010: Nominada al David di Donatello: Film ex...,"6,1",22.498,https://pics.filmaffinity.com/a_serious_man-60...
85,Malditos bastardos,Inglourious Basterds,2009,146 min.,Estados Unidos,Quentin Tarantino,"[Brad Pitt, Christoph Waltz, Mélanie Laurent, ...","[Bélico, Acción, Comedia, II Guerra Mundial, N...",Segunda Guerra Mundial (1939-1945). En la Fran...,[2009: Oscar: Mejor actor de reparto (Christop...,"7,8",150.653,https://pics.filmaffinity.com/inglourious_bast...
86,Precious,Precious: Based on the Novel Push by Sapphire,2009,105 min.,Estados Unidos,Lee Daniels,"[Gabourey Sidibe, Mo'Nique, Paula Patton, Mari...","[Drama, Basado en hechos reales, Cine independ...",Clareece 'Precious' Jones (Gabourey Sidibe) es...,"[2009: 2 Oscars: Mejor guión adaptado, actriz ...","6,7",30.842,https://pics.filmaffinity.com/precious_based_o...
87,Up,Up,2009,96 min.,Estados Unidos,Pete Docter,[],"[Animación, Aventuras, Comedia, Infantil, Amis...",Carl Fredricksen es un viudo vendedor de globo...,"[2009: 2 Oscars: Mejor película de animación, ...","7,9",139.261,https://pics.filmaffinity.com/up-672315222-lar...
88,Up in the Air,Up in the Air,2009,109 min.,Estados Unidos,Jason Reitman,"[George Clooney, Vera Farmiga, Anna Kendrick, ...","[Comedia, Drama, Comedia dramática, Trabajo/em...",A Ryan Bingham (George Clooney) lo contratan l...,"[2009: Oscars: 6 nominaciones, incluyendo mejo...","6,5",50.754,https://pics.filmaffinity.com/up_in_the_air-14...


In [46]:
df.to_csv('movie_data.csv', encoding='utf-8-sig')

In [57]:
for image_url in df["poster"]:
    load_requests(image_url)

C:/Users/a091b/images_FA/green_book-162451488-large.jpg
C:/Users/a091b/images_FA/the_shape_of_water-856013521-large.jpg
C:/Users/a091b/images_FA/moonlight-232276883-large.jpg
C:/Users/a091b/images_FA/spotlight-726888740-large.jpg
C:/Users/a091b/images_FA/birdman_or_the_unexpected_virtue_of_ignorance-402510071-large.jpg
C:/Users/a091b/images_FA/12_years_a_slave_twelve_years_a_slave-305655779-large.jpg
C:/Users/a091b/images_FA/argo-756723442-large.jpg
C:/Users/a091b/images_FA/the_artist-136693699-large.jpg
C:/Users/a091b/images_FA/the_king_s_speech-997653906-large.jpg
C:/Users/a091b/images_FA/the_hurt_locker-853070621-large.jpg
C:/Users/a091b/images_FA/black_panther-992613805-large.jpg
C:/Users/a091b/images_FA/blackkklansman-928120403-large.jpg
C:/Users/a091b/images_FA/bohemian_rhapsody-748186150-large.jpg
C:/Users/a091b/images_FA/the_favourite-984520950-large.jpg
C:/Users/a091b/images_FA/roma-210858899-large.jpg
C:/Users/a091b/images_FA/a_star_is_born-447479418-large.jpg
C:/Users/a091b/